In [14]:
import os
from os import listdir
from os.path import isfile, join
import sys
import pandas as pd

probs_path = '/home/ubuntu/data/d03_classification/probabilities/'
#probs_path = '../probabilities/'
sys.path.append(probs_path)

In [2]:
text_files = [f for f in listdir(probs_path) if isfile(join(probs_path,f))]
print(text_files)

## TODO: make sure we get study / image info. Join accordingly for ground truth

['view_23_e5_class_11-Mar-2018_dicomsample_probabilities.txt']


In [3]:
f = text_files[0]

infile = open(probs_path + f)
infile = infile.readlines()
infile = [i.rstrip() for i in infile]
infile = [i.split('\t') for i in infile]

In [5]:
max_vals = []
max_inds = []
# if probability < 0.01, set to 0
for i in range(len(infile)):
    for j in range(len(infile[0])):
        val = infile[i][j]
        #print(val)
        if i!=0 and j!=0 and j!=1:
            if float(val) < 0.01:
                infile[i][j] = 0.0
    if i!=0:
        row_probs = list(map(float, infile[i][2:]))
        max_vals.append(max(row_probs))
        max_inds.append(row_probs.index(max(row_probs)))
# print(max_vals)
#print(max_inds)

In [6]:
df = pd.DataFrame(infile)
# df = pd.DataFrame(all_probs_list)

df.columns = infile[0]
df.drop(0, inplace=True)
df.drop(labels='study', axis=1, inplace=True)
print(df.shape)

df.head()

(10250, 24)


,image,plax_far,plax_plax,plax_laz,psax_az,psax_mv,psax_pap,a2c_lvocc_s,a2c_laocc,a2c,...,a4c_laocc,a4c,a5c,other,rvinf,psax_avz,suprasternal,subcostal,plax_lac,psax_apex
1,a_85903_AEXT2OYW_114.jpg.dcm,0,0,0,0,0,0,0,0,0,...,0,0.792,0.133,0.021,0,0,0,0,0,0
2,a_113645_3X47N7ME_52.jpg.dcm,0,0.452,0,0,0,0,0,0,0,...,0,0,0,0.514,0,0,0,0.019,0,0
3,a_62720_3UT117D0_19.jpg.dcm,0,0.948,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.043,0
4,a_60896_FONQSE5G_77.jpg.dcm,0,0.015,0,0.06,0.01,0.037,0,0.133,0.093,...,0.023,0.014,0.047,0.215,0.047,0.019,0.173,0.025,0.019,0.011
5,a_105686_BR6HDNSR_21.jpg.dcm,0,0,0,0,0,0,0.099,0,0.799,...,0,0,0,0.087,0,0,0,0,0,0


In [7]:
# parse long string name into study, instance
fn_list = list(df['image'].tolist())
study_list = []
inst_list = []
for fn in fn_list:
    sub = fn.split('_')
    study_list.append(sub[1])
    subsub = sub[2].split('.')
    inst_list.append(subsub[0])

df['study'] = study_list
df['instancefilename'] = inst_list

# rearrange columns, drop image col
cols = df.columns.tolist()
cols_new = cols[-2:] + cols[1:-2]

df_new = df[cols_new]

print(len(df_new['instancefilename'].unique().tolist()))
print(df_new.shape)
df_new.head()

1025
(10250, 25)


,study,instancefilename,plax_far,plax_plax,plax_laz,psax_az,psax_mv,psax_pap,a2c_lvocc_s,a2c_laocc,...,a4c_laocc,a4c,a5c,other,rvinf,psax_avz,suprasternal,subcostal,plax_lac,psax_apex
1,85903,AEXT2OYW,0,0,0,0,0,0,0,0,...,0,0.792,0.133,0.021,0,0,0,0,0,0
2,113645,3X47N7ME,0,0.452,0,0,0,0,0,0,...,0,0,0,0.514,0,0,0,0.019,0,0
3,62720,3UT117D0,0,0.948,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.043,0
4,60896,FONQSE5G,0,0.015,0,0.06,0.01,0.037,0,0.133,...,0.023,0.014,0.047,0.215,0.047,0.019,0.173,0.025,0.019,0.011
5,105686,BR6HDNSR,0,0,0,0,0,0,0.099,0,...,0,0,0,0.087,0,0,0,0,0,0


In [111]:

# print(cols)

In [8]:
cols_view = df_new.columns.tolist()[2:]
view_pred = []

for ind in max_inds:
    view_pred.append(cols_view[ind])

10250


In [9]:
df_new['view_pred'] = view_pred
df_new.head()

,study,instancefilename,plax_far,plax_plax,plax_laz,psax_az,psax_mv,psax_pap,a2c_lvocc_s,a2c_laocc,...,a4c,a5c,other,rvinf,psax_avz,suprasternal,subcostal,plax_lac,psax_apex,view_pred
1,85903,AEXT2OYW,0,0,0,0,0,0,0,0,...,0.792,0.133,0.021,0,0,0,0,0,0,a4c
2,113645,3X47N7ME,0,0.452,0,0,0,0,0,0,...,0,0,0.514,0,0,0,0.019,0,0,other
3,62720,3UT117D0,0,0.948,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.043,0,plax_plax
4,60896,FONQSE5G,0,0.015,0,0.06,0.01,0.037,0,0.133,...,0.014,0.047,0.215,0.047,0.019,0.173,0.025,0.019,0.011,other
5,105686,BR6HDNSR,0,0,0,0,0,0,0.099,0,...,0,0,0.087,0,0,0,0,0,0,a2c


In [15]:
# Add views to join
sys.path.append('../src/d00_utils/')
from db_utils import dbReadWriteData

In [20]:
io_views = dbReadWriteData()
io_views.schema = 'views'
labels_df = io_views.get_table('instances_w_labels')

labels_df.rename(columns={'filename':'instancefilename'}, inplace=True)

labels_df.head()

,sopinstanceuid,instancefilename,instanceidk,view,studyidk
0,1.2.840.113663.1500.1.365900332.3.4.20120703.9...,8Z0BWX0M,696109,plax,39531
1,1.2.840.113663.1500.1.365900332.3.7.20120703.9...,8Z0BWZGH,696112,a4c,39531
2,1.2.840.113663.1500.1.365900332.3.1.20120703.9...,8Z0BXKCB,696140,plax,39533
3,1.2.840.113663.1500.1.365900332.3.14.20120703....,8Z0BXYUW,696157,plax,39533
4,1.2.840.113663.1500.1.365900332.3.8.20120704.9...,8ZGVB44R,696422,plax,39552


In [73]:
# merge tables
merge_df = df_new.merge(labels_df, on='instancefilename')
print(merge_df.shape)
merge_df.rename(index=str, columns={"view": "view_true"}, inplace=True)

# reorder columns for easy analysis
cols = merge_df.columns.tolist()
#print(cols)
cols_new = cols[0:2] + cols[-2:-1] + cols[-5:-4] + cols[2:-5] + cols[-3:-2] + cols[-4:-3]
merge_df = merge_df[cols_new]

cols_all = list(merge_df.columns.values)
cols_probs = cols_all[4:-2]
merge_df[cols_probs] = merge_df[cols_probs].apply(pd.to_numeric)

merge_df.head()

(10250, 30)


,study,instancefilename,view_true,view_pred,plax_far,plax_plax,plax_laz,psax_az,psax_mv,psax_pap,...,a5c,other,rvinf,psax_avz,suprasternal,subcostal,plax_lac,psax_apex,instanceidk,sopinstanceuid
0,85903,AEXT2OYW,a4c,a4c,0.0,0.0,0.0,0.0,0.0,0.0,...,0.133,0.021,0.000,0.0,0.0,0.0,0.0,0.0,1846331,1.2.840.113663.1500.1.406857159.3.59.20160816....
1,85903,AEXT2OYW,a4c,a4c,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,1846331,1.2.840.113663.1500.1.406857159.3.59.20160816....
2,85903,AEXT2OYW,a4c,a4c,0.0,0.0,0.0,0.0,0.0,0.0,...,0.403,0.000,0.000,0.0,0.0,0.0,0.0,0.0,1846331,1.2.840.113663.1500.1.406857159.3.59.20160816....
3,85903,AEXT2OYW,a4c,a4c,0.0,0.0,0.0,0.0,0.0,0.0,...,0.064,0.026,0.027,0.0,0.0,0.0,0.0,0.0,1846331,1.2.840.113663.1500.1.406857159.3.59.20160816....
4,85903,AEXT2OYW,a4c,a2c,0.0,0.0,0.0,0.0,0.0,0.0,...,0.285,0.000,0.000,0.0,0.0,0.0,0.0,0.0,1846331,1.2.840.113663.1500.1.406857159.3.59.20160816....


In [206]:
# average probabilities over ten frames in an instance
cols_groupby = ['instancefilename', 'study', 'view_true']
agg_df = merge_df.groupby(cols_groupby)[cols_probs].mean()
agg_df = agg_df.reset_index(drop=False)

# predicted view is the column with maximum probability
agg_df['view_pred'] = agg_df[cols_probs].idxmax(axis=1)

# reorder columns for visual inspection
cols = agg_df.columns.tolist()
cols_new = cols[0:3] + cols[-1:] + cols[3:-1]
agg_df = agg_df[cols_new]

print(agg_df.shape)
agg_df.head()

(1025, 27)


,instancefilename,study,view_true,view_pred,plax_far,plax_plax,plax_laz,psax_az,psax_mv,psax_pap,...,a4c_laocc,a4c,a5c,other,rvinf,psax_avz,suprasternal,subcostal,plax_lac,psax_apex
0,1BE9ATZO,92634,plax,a3c_laocc,0.0717,0.1231,0.0012,0.0,0.0,0.0057,...,0.1218,0.0000,0.0065,0.1442,0.1202,0.0156,0.0154,0.0554,0.0635,0.0197
1,1BE9I1XJ,92651,a4c,a4c_laocc,0.0000,0.0000,0.0000,0.0,0.0,0.0000,...,0.9997,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,1BUMO8FF,74581,a4c,a4c,0.0000,0.0000,0.0000,0.0,0.0,0.0000,...,0.0000,0.9998,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,1BUMWSJ2,74613,plax,other,0.0000,0.0000,0.0000,0.0,0.0,0.0000,...,0.0000,0.0000,0.0000,0.9985,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,1BUSVDRS,92681,a2c,a2c_laocc,0.0000,0.0000,0.0000,0.0,0.0,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0022,0.0000


In [207]:
# Map b/w view_true, view_pred

#view_pred_unique = list(set(comp_df['view_pred'].tolist()))

# Define a loose mapping from 23-class model to 4-class prediction
# e.g. can map e.g. all a2c --> 'a2c'
plax_views = ['plax_plax', 'plax_lac', 'plax_far', 'plax_laz']
a4c_views = ['a4c', 'a4c_laocc', 'a4c_lvocc_s']
a2c_views = ['a2c_lvocc_s', 'a2c', 'a2c_laocc']
other_views = ['a3c_laocc', 'rvinf', 'a3c', 'suprasternal', \
               'psax_az', 'a3c_lvocc_s', 'other', 'psax_avz', \
               'subcostal', 'a5c']
maps_m1 = {'plax': plax_views, 
           'a4c': a4c_views,
           'a2c': a2c_views,
           'other': other_views
          }

# Define a more strict mapping from 23-class model to 4-class prediction
# e.g. only those classified as pure a2c will be used for measurements
other_views_m2_m3 = other_views + ['plax_lac', 'plax_far', \
                'plax_laz', 'a4c_laocc', 'a4c_lvocc_s', \
                'a2c_lvocc_s', 'a2c_laocc']
maps_m2_m3 = {'plax': ['plax_plax'], 'a2c': ['a2c'], \
              'a4c': ['a4c'], 'other': other_views_m2_m3}

In [208]:
# map prediction of 23 classes into prediction of 4 classes
# NOTE: may be missing some of the original 23 if run on new data
corr_df = agg_df
view23_pred = corr_df['view_pred'].tolist()
view4_pred = []
for pred in view23_pred:
#     view4_pred.append([key for key, element in maps_m1.items() if pred in element][0])
    view4_pred.append([key for key, element in maps_m2_m3.items() if pred in element][0])
corr_df['view4_pred'] = view4_pred

# Add column to compare whether prediction is true or not
corr_df['correct'] = corr_df['view_true'] == corr_df['view4_pred']

# Reorder columns for visual inspection
cols = corr_df.columns.tolist()
cols_new = cols[0:2] + cols[-1:] + cols[2:3] + cols[-2:-1] + cols[3:-2]
corr_df = corr_df[cols_new]
corr_df.rename(columns={'view_pred':'view23_pred'}, inplace=True)

corr_df.head()
## TO DO: save this to a new schema 'results' !

,instancefilename,study,correct,view_true,view4_pred,view23_pred,plax_far,plax_plax,plax_laz,psax_az,...,a4c_laocc,a4c,a5c,other,rvinf,psax_avz,suprasternal,subcostal,plax_lac,psax_apex
0,1BE9ATZO,92634,False,plax,other,a3c_laocc,0.0717,0.1231,0.0012,0.0,...,0.1218,0.0000,0.0065,0.1442,0.1202,0.0156,0.0154,0.0554,0.0635,0.0197
1,1BE9I1XJ,92651,False,a4c,other,a4c_laocc,0.0000,0.0000,0.0000,0.0,...,0.9997,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,1BUMO8FF,74581,True,a4c,a4c,a4c,0.0000,0.0000,0.0000,0.0,...,0.0000,0.9998,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,1BUMWSJ2,74613,False,plax,other,other,0.0000,0.0000,0.0000,0.0,...,0.0000,0.0000,0.0000,0.9985,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,1BUSVDRS,92681,False,a2c,other,a2c_laocc,0.0000,0.0000,0.0000,0.0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0022,0.0000


In [212]:
#io_views.save_to_db(corr_df, 'output_m1_test1000_aug8')

In [209]:
print(corr_df['view_true'].value_counts())
print(corr_df['view4_pred'].value_counts())

plax    503
a4c     311
a2c     211
Name: view_true, dtype: int64
other    517
plax     274
a4c      148
a2c       86
Name: view4_pred, dtype: int64


In [211]:
### EVALUATION WITH STRICT MAPPING
# Eval throwing out prediction of 'other' view

df = corr_df
df = df[df['view4_pred'] != 'other']
print(df['correct'].value_counts())
# 508 / 1025 = 50% of all instances are NOT predicted as other
# On those instances, accuracy of 500/508 = 97%

pd.crosstab(df['correct'], [df['view_true']])
# X/Y on [true view]
# 274274 on plax, 145/149 on a4c, 81/84 on a2c

pd.crosstab(df['correct'], [df['view_true'], df['view4_pred']])

True     500
False      8
Name: correct, dtype: int64


view_true  a2c     a4c      plax
view4_pred a2c a4c a2c  a4c plax
correct                         
False        0   3   5    0    0
True        81   0   0  145  274

In [194]:
### EVALUATION WITH LOOSE MAPPING
# Eval throwing out prediction of 'other' view

df = corr_df
df = df[df['view4_pred'] != 'other']
print(df['correct'].value_counts())
# 768 / 1025 = 75% of all instances are NOT predicted as other
# On those instances, accuracy of 748 / 768 = 97%

pd.crosstab(df['correct'], [df['view_true']])
# X/Y on [true view]
# 353/355 on plax, 243/253 on a4c, 152/160 on a2c

pd.crosstab(df['correct'], [df['view_true'], df['view4_pred']])

True     748
False     20
Name: correct, dtype: int64


view_true   a2c     a4c      plax     
view4_pred  a2c a4c a2c  a4c  a4c plax
correct                               
False         0   8  10    0    2    0
True        152   0   0  243    0  353

In [196]:
df = corr_df
print(df['correct'].value_counts())

pd.crosstab(df['correct'], [df['view_true'], df['view4_pred']])

True     748
False    277
Name: correct, dtype: int64


view_true   a2c           a4c            plax           
view4_pred  a2c a4c other a2c  a4c other  a4c other plax
correct                                                 
False         0   8    51  10    0    58    2   148    0
True        152   0     0   0  243     0    0     0  353

In [160]:
df = comp_df
other_df = df[df['view4_pred'] == 'other']

print(other_df['correct'].value_counts())

False    257
Name: correct, dtype: int64


In [163]:
lab_df = df[df['view4_pred'] != 'other']

print(lab_df['correct'].value_counts())
print()
print(lab_df['view_true'].value_counts())

False    542
True     226
Name: correct, dtype: int64

plax    355
a4c     253
a2c     160
Name: view_true, dtype: int64


In [105]:
print(df['view_true'].value_counts())
print(df['view_pred'].value_counts())

plax    503
a4c     311
a2c     211
Name: view_true, dtype: int64
plax_plax       274
other           173
a4c             148
a4c_laocc       104
a2c              86
a2c_laocc        72
plax_lac         63
psax_az          20
a3c              19
psax_avz         15
a3c_laocc        15
plax_laz         10
plax_far          6
a5c               5
rvinf             4
a2c_lvocc_s       4
subcostal         4
a3c_lvocc_s       1
a4c_lvocc_s       1
suprasternal      1
Name: view_pred, dtype: int64


In [101]:
df_incorrect = df[df['correct']==False]
df = df_incorrect
print(df['view_pred'].value_counts())

plax_plax       274
other           173
a4c_laocc       104
a2c_laocc        72
plax_lac         63
psax_az          20
a3c              19
a3c_laocc        15
psax_avz         15
plax_laz         10
plax_far          6
a5c               5
a2c               5
a2c_lvocc_s       4
rvinf             4
subcostal         4
a4c               3
a3c_lvocc_s       1
suprasternal      1
a4c_lvocc_s       1
Name: view_pred, dtype: int64


['plax_plax', 'plax_lac', 'plax_far', 'plax_laz']
['plax_plax', 'plax_lac', 'plax_far', 'plax_laz']
['plax_plax', 'plax_lac', 'plax_far', 'plax_laz']
['plax_plax', 'plax_lac', 'plax_far', 'plax_laz']
['a3c_laocc', 'rvinf', 'a3c', 'suprasternalpsax_az', 'a3c_lvocc_s', 'other', 'psax_avz']
['a3c_laocc', 'rvinf', 'a3c', 'suprasternalpsax_az', 'a3c_lvocc_s', 'other', 'psax_avz']
['a3c_laocc', 'rvinf', 'a3c', 'suprasternalpsax_az', 'a3c_lvocc_s', 'other', 'psax_avz']
['a3c_laocc', 'rvinf', 'a3c', 'suprasternalpsax_az', 'a3c_lvocc_s', 'other', 'psax_avz']
['a4c', 'a4c_laocc', 'a4c_lvocc_s']
['a4c', 'a4c_laocc', 'a4c_lvocc_s']
['a4c', 'a4c_laocc', 'a4c_lvocc_s']
['a4c', 'a4c_laocc', 'a4c_lvocc_s']


In [86]:
monty = { 
    ('parrot','spam','cheese_shop'): 'sketch', 
    ('Cleese', 'Gilliam', 'Palin') : 'actors',
}

print(monty['parrot'])

KeyError: 'parrot'

In [82]:
# For each row, get column with highest probability

cols_probs = merge_df.columns.tolist()[3:-2]

df_probs = merge_df[cols_probs]
df_probs['max_prob'] = df_probs.max(axis=1)

df_probs.head()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,plax_far,plax_plax,plax_laz,psax_az,psax_mv,psax_pap,a2c_lvocc_s,a2c_laocc,a2c,a3c_lvocc_s,...,a4c,a5c,other,rvinf,psax_avz,suprasternal,subcostal,plax_lac,psax_apex,max_prob
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00020000001,0.0,0.0,...,0.0018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9973
1,0.030699998,0.0006,0.0007000001,0.00040000002,0.0174,0.30920002,0.0006,0.0172,0.0006,0.0056000003,...,0.000100000005,0.0,0.22569999,0.0492,0.0007999999,0.017,0.1007,0.015700001,0.1911,0.3092
2,0.0,0.0,0.0,0.0,0.0,0.000100000005,0.0,0.0,0.0,0.0,...,0.0,0.0,0.9992,0.0,0.0,0.0,0.0,0.00020000001,0.0,0.9992
3,0.0003,0.00080000004,0.0003,0.0025,0.0003,0.000100000005,0.0084,0.0048,0.49709997,0.00070000003,...,0.4112,0.0144,0.0313,0.0061,0.0007999999,0.0012,0.0006,0.0017000001,0.0009,0.4971
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0072,0.18099998,0.19860001,0.0,...,0.0003,0.0003,0.60800004,0.00020000001,0.0,0.0,0.0,0.00080000004,0.000100000005,0.6080


In [84]:
print(cols_probs)

['plax_far', 'plax_plax', 'plax_laz', 'psax_az', 'psax_mv', 'psax_pap', 'a2c_lvocc_s', 'a2c_laocc', 'a2c', 'a3c_lvocc_s', 'a3c_laocc', 'a3c', 'a4c_lvocc_s', 'a4c_laocc', 'a4c', 'a5c', 'other', 'rvinf', 'psax_avz', 'suprasternal', 'subcostal', 'plax_lac', 'psax_apex']


In [92]:
for col in cols_probs[0:3]:
#     print(df_probs[col])
    if (df_probs[col].any() == df_probs['max_prob']).any():
        print(col)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
